In [1]:
from transformers import BertTokenizer, BertModel
import torch
import torch.nn.functional as F

In [2]:
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")
model = BertModel.from_pretrained("bert-base-uncased")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

c:\Users\Lenovo\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Lenovo\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [3]:
query = "How many apples are there on the table?"
sentence1 = "Three apples are lying on the table."
sentence2 = "The table depicts the sale of apples in 2024."

In [4]:
def get_token_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)
    
    with torch.no_grad():
        outputs = model(**inputs)
        
    # Shape: [num_tokens, hidden_dim]
    token_embeddings = outputs.last_hidden_state.squeeze(0) 
    
    return token_embeddings, inputs["input_ids"]

query_embeddings, query_input_ids = get_token_embeddings(query)
sentence1_embeddings, sentence1_input_ids = get_token_embeddings(sentence1)
sentence2_embeddings, sentence2_input_ids = get_token_embeddings(sentence2)

In [5]:
def match_score(query_embeddings, sentence_embeddings):

    # Shape: [num_query_tokens, num_sentence_tokens]
    similarity_matrix = torch.matmul(query_embeddings,
                                     sentence_embeddings.T)
    
    # Max similarity for each query token
    max_scores, _ = torch.max(similarity_matrix, dim=1)
    
    # Aggregate scores
    return max_scores.sum().item()

In [6]:
score1 = match_score(query_embeddings, sentence1_embeddings)
score2 = match_score(query_embeddings, sentence2_embeddings)

if score1 > score2:
    print("Sentence 1 is more relevant to the query.")
else:
    print("Sentence 2 is more relevant to the query.")
    


Sentence 1 is more relevant to the query.


In [7]:
query = "How many apples are there on the table?"

sentence1 = ("Three apples are lying on the table. "
            "Alongside, there are three oranges and three bananas. "
            "There is also a tablecloth on the table. "
            "There is a table lamp on the table. "
            "There are two people sitting on the table.")


sentence2 = ("The table depicts the sale of apples in 2024. "
            "This table was produced by our company. "
            "This table also portrays the sale of oranges in 2024. "
            "Lastly, this table can be used to calculate the total revenue.")

In [8]:
query_embeddings, query_input_ids = get_token_embeddings(query)
sentence1_embeddings, sentence1_input_ids = get_token_embeddings(sentence1)
sentence2_embeddings, sentence2_input_ids = get_token_embeddings(sentence2)

score1 = match_score(query_embeddings, sentence1_embeddings)
score2 = match_score(query_embeddings, sentence2_embeddings)

if score1 > score2:
    print("Sentence 1 is more relevant to the query.")
else:
    print("Sentence 2 is more relevant to the query.")

Sentence 1 is more relevant to the query.
